<a href="https://colab.research.google.com/github/HCCREN/whisper_voice/blob/main/WHISPER_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=9999d2a0cff9e9a7c3e1fb0e9dfc2f0f4c7cfc011737dd55b455fe01ef8afed1
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
!pip install pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pydub import AudioSegment
import torch
import whisper
def m4a_to_wav(input_path, output_path):
    audio = AudioSegment.from_file(input_path, "m4a")
    audio.export(output_path, format="wav")

def split_audio(audio, interval_length_ms, output_folder):
    # Calculate the number of segments needed
    num_segments = len(audio) // interval_length_ms

    # Split the audio
    for i in range(num_segments):
        start_time = i * interval_length_ms
        end_time = (i + 1) * interval_length_ms
        segment = audio[start_time:end_time]

        # Export each segment
        segment.export(f"{output_folder}/segment_{i+1}.wav", format="wav")

'''file_name = 'Sato1.m4a'
input_file_path = '/content/drive/MyDrive/Colab Notebooks/sound_data/' + file_name  # Replace with your M4A file path
output_file_path = '/content/drive/MyDrive/Colab Notebooks/sound_data/' + file_name.split('.')[0] + '.wav'  # Output WAV file path
output_folder = '/content/drive/MyDrive/Colab Notebooks/sound_data/split_segments'  # Output folder for the split segments'''
output_file_path = '/content/drive/MyDrive/Colab Notebooks/sound_data/news.wav'
output_folder = '/content/drive/MyDrive/Colab Notebooks/sound_data/split_segments'
# Convert M4A to WAV
#m4a_to_wav(input_file_path, output_file_path)

# Load the converted WAV file
#audio = AudioSegment.from_wav(output_file_path)
audio = AudioSegment.from_wav(output_file_path)
# Interval length in milliseconds (e.g., 1000ms = 1s)
interval_length_ms = 2640000

# Split and save the audio segments
split_audio(audio, interval_length_ms, output_folder)

In [ ]:

# Check if CUDA is available and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the whisper model
model_size = "large-v3"  # or "small", "medium", "large" depending on the model size you want to load
model = whisper.load_model(model_size).to(device)

def transcribe_audio_files(folder_path, model, output_folder):
    # List all the files in the folder
    file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    for file_name in file_names:
        if not file_name.endswith('.wav'):
            continue  # Skip non-WAV files

        # Load audio
        audio_path = os.path.join(folder_path, file_name)
        audio = whisper.load_audio(audio_path)

        # Transcribe audio
        text = model.transcribe(audio)

        # Create and save text file
        output_text_path = os.path.join(output_folder, f"{file_name.split('.')[0]}_transcription.txt")
        with open(output_text_path, "w") as f:
            f.write(text['text'])

# Folder containing the audio segments
input_folder = '/content/drive/MyDrive/Colab Notebooks/sound_data/split_segments'

# Output folder for the transcriptions
output_folder = '/content/drive/MyDrive/Colab Notebooks/sound_data/transcriptions'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Transcribe and save text
transcribe_audio_files(input_folder, model, output_folder)


100%|█████████████████████████████████████| 2.88G/2.88G [00:40<00:00, 75.9MiB/s]
